## Simple Starter Code: pytorch + EfficientNet 

In [ ]:
# !pip install -q efficientnet_pytorch
# from efficientnet_pytorch import EfficientNet

#install efficientnet without internet
!mkdir -p /tmp/pip/cache/
!cp ../input/resources-for-google-landmark-recognition-2020/efficientnet_pytorch-0.6.3-py3-none-any.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ efficientnet_pytorch

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler
import pytorch_lightning as pl
from tqdm import tqdm
import albumentations as A
from efficientnet_pytorch import EfficientNet

from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,accuracy_score
from sklearn.model_selection import train_test_split,StratifiedKFold,GroupKFold,KFold
import pandas as pd
import numpy as np 
import gc
import os
import glob
from collections import defaultdict
import random
import cv2
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

import warnings
import matplotlib.pyplot as plt
from matplotlib import rc
from pylab import rcParams
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

In [ ]:
#lets checkout the train data
label_to_class = {
0:"Cassava Bacterial Blight (CBB)",
1:"Cassava Brown Streak Disease (CBSD)",
2:"Cassava Green Mottle (CGM)",
3:"Cassava Mosaic Disease (CMD)",
4:"Healthy",
}
df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
sns.countplot(df.label.map(label_to_class))
plt.xticks(rotation=30)
plt.title('Target Distribution')

In [ ]:
#utilities to load and display images
def load_image(img_path,resize=True):
    img = cv2.cvtColor(cv2.imread(img_path),cv2.COLOR_BGR2RGB)
    if resize:
        img = cv2.resize(img, (224,224),interpolation=cv2.INTER_AREA)
    return img

def show_img(img_path):
    img = load_image(img_path)
    plt.imshow(img)
    plt.axis('off')
    

def show_img_grid(img_path):
    images = [load_image(img) for img in img_path]
    images = torch.as_tensor(images)
    images = images.permute(0,3,1,2)
    grid_img = torchvision.utils.make_grid(images,nrow=5)
    plt.figure(figsize=(25,12))
    plt.imshow(grid_img.permute(1,2,0))
    plt.axis('off')

In [ ]:
IMG_PATH = '../input/cassava-leaf-disease-classification/train_images/' 
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images/'
path = [os.path.join(IMG_PATH,img) for img in df.loc[:24,'image_id']]
show_img_grid(path)

In [ ]:
class LeafDiseaseDataset(Dataset):
    def __init__(self, img_path: str,targets,transforms = None, resize=None,test=False):
        self.img_path = img_path
        self.targets = targets
        self.transforms = transforms
        self.resize = resize
        self.test = test
        
    def __len__(self):
        return len(self.img_path)
        
    def __getitem__(self, item):
        image = Image.open(self.img_path[item])
        image = image.convert('RGB')
        if not self.test:
            targets = self.targets[item]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1],self.resize[0]),
                resample=Image.BILINEAR
            )
        image = np.array(image)
        
        if self.transforms is not None:
            augmented = self.transforms(image=image)
            image = augmented['image']
            
        image = np.transpose(image,(2,0,1)).astype(np.float32)        
        if not self.test:
            return {
                "image": torch.tensor(image, dtype=torch.float),
                "targets": torch.tensor(targets, dtype=torch.long),
            }
        else:
            return {
                "image": torch.tensor(image, dtype=torch.float),
            }
            

In [ ]:
class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        self.base = EfficientNet.from_name(f'efficientnet-b4')
        pretrained_file = '../input/resources-for-google-landmark-recognition-2020/efficientnet-b4-6ed6700e.pth'
        self.base.load_state_dict(torch.load(pretrained_file))
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.output_filter = self.base._fc.in_features
        self.classifier = nn.Linear(self.output_filter, num_classes)
    def forward(self, x):
        x = self.base.extract_features(x)
        x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x)
        return x
    
#ResNet-50
class LeafResNet(nn.Module):
    def __init__(self, resnet, n_targets:int):
        super(LeafResNet,self).__init__()
        self.resnet = resnet
        for param in resnet.parameters():
            param.requires_grad = False
        resnet.fc = nn.Sequential(
            nn.Linear(2048,128),
            nn.ReLU(inplace=True),
            nn.Linear(128, n_targets)
        )
    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
#we have aleady seen that the data is bit imbalanced so we can perfrom sratifiedKfold
kf = StratifiedKFold(n_splits=5)
df['kfolds'] = -1
for fold_,(t_,v_) in enumerate(kf.split(X=df,y=df.label.values)):
    df.loc[v_,'kfolds'] = fold_
sns.countplot(df.kfolds)

In [ ]:
#train/eval step
def train(data_loader, model, 
          loss_fn, optimizer, 
          scheduler,device,n_examples):
    model.train()
    losses = []
    accs = []
    for data in data_loader:
        correct_predictions = 0
        inputs = data['image'].to(device,dtype=torch.float)
        targets = data['targets'].to(device,dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        _,preds = torch.max(outputs,dim=1)
        loss = loss_fn(outputs,targets)
        accuracy = accuracy_score(targets.cpu().numpy(), preds.cpu().numpy())
        
        correct_predictions += torch.sum(preds==targets).cpu().numpy()
        losses.append(loss.item())
        accs.append(accuracy)
        
        loss.backward()
        optimizer.step()
    scheduler.step()

    return np.mean(accs), np.mean(losses)
        
def evaluate(data_loader, model,
             loss_fn, device, n_examples):
    
    model.eval()
    losses = []
    accs = []
    correct_predictions = 0
    with torch.no_grad():
        for data in data_loader:
            inputs = data['image'].to(device,dtype=torch.float)
            targets = data['targets'].to(device,dtype=torch.long)
            outputs = model(inputs)
            _,preds = torch.max(outputs,dim=1)
            loss = loss_fn(outputs, targets)
            
            accuracy = accuracy_score(targets.cpu().numpy(), preds.cpu().numpy())
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
            accs.append(accuracy)

    return np.mean(accs), np.mean(losses)

In [ ]:
#config settings
data_path = '../input/cassava-leaf-disease-classification/train_images/'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"device: {device}")
N_epochs = 10
batch_size=64
num_workers=4
model_arch = 'tf_efficientnet_b4_ns'

mean = (0.485,0.456,0.406)
std = (0.229,0.224,0.225)
train_aug = A.Compose(
[
    A.ShiftScaleRotate(shift_limit=0.05,scale_limit=0.05,rotate_limit=15,p=0.5),
    A.HorizontalFlip(),
    A.RandomRotate90(),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean,std,max_pixel_value=255.0,always_apply=True)
])

val_aug = A.Compose([
    A.Normalize(mean,std,max_pixel_value=255.0,always_apply=True)
])


In [ ]:
def train_model(model,train_data_loader,
                val_data_loader,
                device,n_epochs):
    optimizer = optim.Adam(model.parameters(),lr=1e-3)
    scheduler = lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)
    loss_fn = nn.CrossEntropyLoss().to(device)
    
    history = defaultdict(list)
    best_accuracy = 0
    
    for epoch in range(n_epochs):
        print(f"epoch {epoch + 1}/{n_epochs}")
        print('-' * 10)
        train_acc,train_loss = train(
            train_data_loader,model,
            loss_fn,optimizer,
            scheduler,device,len(train_data_loader)
        )
        print(f"Train loss {train_loss}, train_acc: {train_acc}")
        
        val_acc, val_loss = evaluate(
            val_data_loader,model,
            loss_fn,device,len(val_data_loader)
        )
        
        print(f"Val loss: {val_loss}, val accuracy: {val_acc}")
        history['train_acc'].append(train_acc)
        history['train_loss'].append(train_loss)
        history['val_acc'].append(val_acc)
        history['val_loss'].append(val_loss)
        
        if val_acc > best_accuracy:
            torch.save(model.state_dict(), f'best_model_state_{val_acc}.bin')
            best_accuracy = val_acc
        
    print(f"Best val accuracy: {best_accuracy}")
    
    return history,f'best_model_state_{best_accuracy}.bin'

In [ ]:
# Kfold cross validation training
'''for fold_ in range(5):
    model = LeafNet(eff,len(df.label.unique()))
    model.to(device)
    print(f"___________FOLD: {fold_}____________")
    train_df,val_df = df[df.kfolds!=fold_], df[df.kfolds==fold_] 
    train_img_id = train_df.image_id.values.tolist()
    train_imgs = [os.path.join(data_path, img) for img in train_img_id]
    train_targets = train_df.label.values
    
    val_img_id = val_df.image_id.values.tolist()
    val_imgs = [os.path.join(data_path, img) for img in val_img_id]
    val_targets = val_df.label.values
    
    train_dataset = LeafDiseaseDataset(img_path=train_imgs,
                                       targets=train_targets,
                                       transforms=train_aug,
                                       resize=(224,224)
                                      )
    train_loader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)
    
    valid_dataset = LeafDiseaseDataset(img_path=val_imgs,
                                      targets=val_targets,
                                      transforms=val_aug,
                                      resize=(224,224))
    valid_loader = DataLoader(valid_dataset,
                              batch_size=batch_size * 2,
                              num_workers=num_workers)
    
    train_model(model,train_loader,valid_loader,device,N_epochs)'''
    

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
xtrain,xtest,ytrain,ytest = train_test_split(df,df.label,test_size=0.1)
train_img_id = xtrain.image_id.values.tolist()
train_imgs = [os.path.join(data_path,img) for img in train_img_id]
train_targets = ytrain.values

test_img_id = xtest.image_id.values.tolist()
test_imgs = [os.path.join(data_path,img) for img in test_img_id]
test_targets = ytest.values

train_dataset = LeafDiseaseDataset(img_path=train_imgs,
                                    targets=train_targets,
                                    transforms=train_aug,
                                    resize=(224,224)
                                      )
train_loader = DataLoader(train_dataset,
                        batch_size=batch_size,
                        num_workers=num_workers)
    
test_dataset = LeafDiseaseDataset(img_path=test_imgs,
                                targets=test_targets,
                                transforms=val_aug,
                                resize=(224,224))
test_loader = DataLoader(test_dataset,
                              batch_size=batch_size * 2,
                              num_workers=num_workers)
model = EfficientNetEncoderHead(depth=4, num_classes=len(df.label.unique()))
model.to(device)


hist,best_model_pth = train_model(model,train_loader,test_loader,device,N_epochs)

In [ ]:
test_model = EfficientNetEncoderHead(depth=4, num_classes=len(df.label.unique()))
test_model.to(device)
test_model.load_state_dict(torch.load(best_model_pth))

In [ ]:
test_imgs = os.listdir('../input/cassava-leaf-disease-classification/test_images/')
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images/'
img = Image.open(os.path.join(TEST_PATH, test_imgs[0]))
img

In [ ]:
image = img.convert('RGB')
image = image.resize(
        (224,224),
        resample=Image.BILINEAR
    )
image = np.array(image)
img_tfs = val_aug(image=image) 
image = torch.tensor(img_tfs['image'],dtype=torch.float32)

In [ ]:
final_result = []
img = image.permute(2,0,1).unsqueeze(0).to(device)
preds = torch.argmax(test_model(img))
final_result.append([test_imgs[0],preds.item()])

#submission
sub = pd.DataFrame(final_result, columns=['image_id', 'label'])
sub.to_csv('submission.csv',index=False)

In [ ]:
def final_submission(model, dataloader, device, samp_sub):
    model.eval()
    img_preds_all = []
    with torch.no_grad():
        for data in dataloader:
            inputs = data['image'].to(device,dtype=torch.float)
            output = model(img)
            img_preds_all += [torch.softmax(output, 1).detach().cpu().numpy()]

    img_preds_all = np.concatenate(img_preds_all, axis=0)
    samp_sub["label"] = np.argmax(img_preds_all, axis=1)
    samp_sub.to_csv("submission.csv", index=False)

In [ ]:
samp_sub = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
samp_sub_img_id = samp_sub.image_id.values.tolist()
samp_sub_imgs = [os.path.join(TEST_PATH,img) for img in samp_sub_img_id]


samp_sub_dataset = LeafDiseaseDataset(img_path=samp_sub_imgs,
                                    targets=None,
                                    transforms=val_aug,
                                    resize=(224,224),
                                    test = True
                                      )
samp_sub_loader = DataLoader(samp_sub_dataset,
                        batch_size=batch_size,
                        num_workers=num_workers)

final_submission(test_model,samp_sub_loader,device,samp_sub)